In [15]:
# ============================================
# Función Local: validacionME53N
# Autor: Paula Sierra - NetApplications
# Descripcion: Ejecuta ME5A y exporta archivo TXT según estado.
# Ultima modificacion: 24/11/2025
# Propiedad de Colsubsidio
# Cambios: (Si Aplica)
# ============================================
from requests import session
import win32com.client
import traceback
import pandas as pd
import re
import subprocess
import time
import os
from funciones.EscribirLog import WriteLog 
from config.settings import RUTAS
import pyautogui
from pyautogui import ImageNotFoundException
from funciones.Login import ObtenerSesionActiva
from funciones.GuiShellFunciones import (SapTextEditor,
set_GuiTextField_text,              
get_GuiTextField_text,
buscar_objeto_por_id_parcial,
obtener_valor,
extraer_concepto,
obtener_correos,
normalizar_precio_sap, 
clasificar_concepto,
esperar_sap_listo,
buscar_y_clickear, set_sap_table_scroll, 
ventana_abierta,
SelectGuiTab,
MostrarCabecera,
obtener_numero_oc
)
from typing import List, Literal, Optional




def ValidarAjustarSolped(session,item=1):
    """
    Cambia los precios y las cantidades de la Solped segun el texto del "Texto pedido" (textPF.selectedNode ="F01")
    borra los textos adicionales que no se utilizan (textPF.selectedNode ="F02"), hasta el F05

    Args:
        session: sesión SAP activa
        item (int): posiciones que tiene la Solped

    Raises:
        Exception si no se encuentra el objeto
    """
    bandera=True
    while (bandera == True):
        try:
            
            # Lista de acciones en SAP que sirve de informe
            acciones = []

            for fila in range(item):  #cambiar por item
                # Selecbox de la posicion de la solped  ejemplo de guia :  1 [10] 80016676 , LAVADO MANTEL GRANDE 
                PosicionSolped = buscar_objeto_por_id_parcial(session, "cmbDYN_6000-LIST")
                PosicionSolped.key = f"   {fila+1}"
                
                CantidadPosicion = get_GuiTextField_text(session, f"MENGE[6,{fila}]")

                # Navega a la pestaña de textos
                esperar_sap_listo(session)
                SelectGuiTab(session, "TABIDT14")
                textPF = buscar_objeto_por_id_parcial(session, "cntlTEXT_TYPES_0200/shell")
                textPF.selectedNode = "F01"  # Foco en primer Texto IMPORTANTE
                
                esperar_sap_listo(session)
                EDITOR_ID= buscar_objeto_por_id_parcial(session, "cntlTEXT_EDITOR_0201/shellcont/shell")

                esperar_sap_listo(session)
                # obtiene el texto del objeto ├─ Leer textos
                editor = SapTextEditor(session, EDITOR_ID.id)
                texto = editor.get_all_text()
            
                # Obtiene la cantidad en el texto
                claves = ["CANTIDAD"] # str que busca en el texto
                cantidadtexto = obtener_valor(texto, claves)

                # Comparacion de Valores de Cantidad
                if cantidadtexto and cantidadtexto.strip() and CantidadPosicion != cantidadtexto:
                    set_GuiTextField_text(session, f"MENGE[6,{fila}]", cantidadtexto)
                    #print(f"Se mofico posicion :{fila+1}0 Cantidad -> {CantidadPosicion} != {cantidadtexto}")
                    acciones.append(f"Posicion {fila+1}0 => CP: {CantidadPosicion} != CT:{cantidadtexto} ⚠️ Se Actualiza Cantidad")                 

                esperar_sap_listo(session)
            # Devuelve las accines ejecutadas en una lista 
            return acciones

        except Exception as e:
            #todo: canbiar por log
            set_sap_table_scroll(session, "TC_1211", fila)
            print(f"\nHa ocurrido un error inesperado durante la ejecución: {e}")
            raise


session = ObtenerSesionActiva()
acciones = ValidarAjustarSolped(session, 7)
print(acciones)

 Sesion encontrada en conexión: ERP-CORPORATIVO-CALIDAD

Ha ocurrido un error inesperado durante la ejecución: No se encontró GuiTextField: MENGE[6,3]


Exception: No se encontró GuiTextField: MENGE[6,3]

In [ ]:
# ================================
# Main: GestionSolped
# Autor: Paula Sierra, Henry Navarro - NetApplications
# Descripcion: Main principal del Bot
# Ultima modificacion: 24/11/2025
# Propiedad de Colsubsidio
# Cambios: Ajuste inicial para cumplimiento de estándar


# from NetApplications.PY.AutomatizacionGestionSolped.HU.HU03_ValidacionME53N import buscar_SolpedME53N
from funciones.EscribirLog import WriteLog
import traceback
from config.settings import RUTAS,SAP_CONFIG
from funciones.DescargarSolpedME5A import DescargarSolpedME5A
from funciones.GuiShellFunciones import set_sap_table_scroll,get_GuiTextField_text
from funciones.Login import ObtenerSesionActiva
from HU.HU02_DescargaME5A import EjecutarHU02
from funciones.GeneralME53N import procesarTablaME5A
from funciones.GuiShellFunciones import ProcesarTabla


#session = ObtenerSesionActiva()
dataFrame, primera_fila = ProcesarTabla("expSolped05.txt")



print(dataFrame)
print(primera_fila)





EXITO: Columna 'Estado' agregada al DataFrame
EXITO: Columna 'Observaciones' agregada al DataFrame
   - SOLPEDs: 2927
   - Estados unicos: {'': 5703}
INFO: No se aplicó filtro por ReqDate (dias=None)


c:\Users\CGRPA042\Documents\Steven\SOLPED\NetApplications\PY\AutomatizacionGestionSolped\funciones\GuiShellFunciones.py:1128: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ReqDate_fmt"] = pd.to_datetime(
c:\Users\CGRPA042\Documents\Steven\SOLPED\NetApplications\PY\AutomatizacionGestionSolped\funciones\GuiShellFunciones.py:1132: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["ReqDate_fmt"] = pd.to_datetime(


ValueError: too many values to unpack (expected 2)

In [11]:
# ================================
# Main: GestionSolped
# Autor: Paula Sierra, Henry Navarro - NetApplications
# Descripcion: Main principal del Bot
# Ultima modificacion: 24/11/2025
# Propiedad de Colsubsidio
# Cambios: Ajuste inicial para cumplimiento de estándar


# from NetApplications.PY.AutomatizacionGestionSolped.HU.HU03_ValidacionME53N import buscar_SolpedME53N
from funciones.EscribirLog import WriteLog
import pandas as pd
import traceback
from config.settings import RUTAS,SAP_CONFIG
from funciones.DescargarSolpedME5A import DescargarSolpedME5A
from funciones.GuiShellFunciones import set_sap_table_scroll,get_GuiTextField_text
from funciones.Login import ObtenerSesionActiva
from HU.HU02_DescargaME5A import EjecutarHU02
from funciones.GeneralME53N import procesarTablaME5A
#from funciones.GuiShellFunciones import ObtenerColumnasdf



session = ObtenerSesionActiva()

#set_sap_table_scroll(session, "TC_1211", 4)
preuba = get_GuiTextField_text(session, f'EBELP[1,5]')
print(f"Primera posicion visible : {preuba}")
print(type(preuba))


 Sesion encontrada en conexión: ERP-CORPORATIVO-CALIDAD
Primera posicion visible : 
<class 'str'>


In [1]:

# ================================
# Main: GestionSolped
# Autor: Paula Sierra, Henry Navarro - NetApplications
# Descripcion: Main principal del Bot
# Ultima modificacion: 24/11/2025
# Propiedad de Colsubsidio
# Cambios: Ajuste inicial para cumplimiento de estándar


# from NetApplications.PY.AutomatizacionGestionSolped.HU.HU03_ValidacionME53N import buscar_SolpedME53N
from funciones.EscribirLog import WriteLog
import pandas as pd
import traceback
from config.settings import RUTAS,SAP_CONFIG
from funciones.DescargarSolpedME5A import DescargarSolpedME5A
from funciones.GuiShellFunciones import set_sap_table_scroll,get_GuiTextField_text
from funciones.Login import ObtenerSesionActiva
from HU.HU02_DescargaME5A import EjecutarHU02
from funciones.GeneralME53N import procesarTablaME5A
#from funciones.GuiShellFunciones import ObtenerColumnasdf


session = ObtenerSesionActiva()

bandera=True

while (bandera == True):
    try :
        for i in range(20):  # Revisa las condiciones
            impuestosCondiciones = get_GuiTextField_text(session, f"VTEXT[2,{i}]")
            print(f"Impuesto en la pestaña de condiciones: {impuestosCondiciones}")
            #if impuestosCondiciones == "Imp. Saludable IBUE": # cambiar aqui el valor a buscar
            if impuestosCondiciones == "Valor efectivo": 
                print("Impuesto encontrado:", impuestosCondiciones)
                bandera=False
                break
            elif impuestosCondiciones == "" :
                print("no encontrado:", impuestosCondiciones)
                bandera=False
                break

    except Exception as e:
        set_sap_table_scroll(session, "tblSAPLV69ATCTRL_KONDITIONEN", i)
        print("Error al obtener los impuestos de las condiciones:", str(e))
        #continue
        



 Sesion encontrada en conexión: ERP-CORPORATIVO-CALIDAD
Impuesto en la pestaña de condiciones: Precio bruto
Impuesto en la pestaña de condiciones: Valor Bruto
Impuesto en la pestaña de condiciones: AIU UTILIDAD
Impuesto en la pestaña de condiciones: Valor neto incl.desc
Impuesto en la pestaña de condiciones: IVA sop.no deducible
Impuesto en la pestaña de condiciones: Valor neto incl.IVA
Error al obtener los impuestos de las condiciones: No se encontró GuiTextField: VTEXT[2,6]
Impuesto en la pestaña de condiciones: Dto.pronto pago
Impuesto en la pestaña de condiciones: Valor efectivo
Impuesto encontrado: Valor efectivo


In [ ]:
IDENTI= f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ID}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14"    


In [27]:
import pyautogui

a=1
ventana=EncontrarDynpros(session)
ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
Posiciones=session.FindById(ID).Entries.Count
for i in range(Posiciones-2):
    print(i)
    combo=session.FindById(ID)
    item=session.FindById(ID).Entries(i)
    print(combo.Entries(i).value)
    combo.key=item.key
    # lista de posiciones de la solped
    #print(f"Navegando por la posición {i+1} de la Solped {solped}Esperando foco en pestaña textos..")
    # foco Select en la pestaña textos despues de imputacion 
    ventana=EncontrarDynpros(session)     
    ID= f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14"    
    print(f"ID Dinamico es: {ID}")
    session.FindById(ID).select()
    session.FindById(ID).setFocus()
    # ir a texto posicion con un TAB 
    pyautogui.hotkey("TAB")
    time.sleep(0.5)
    pyautogui.press('down') 
    for i in range(5):   
      # ir a pedido info con un flecha abajo
      pyautogui.press('down') 
      # entrar a editar texto con ctrl + enter
      pyautogui.hotkey("ctrl","enter")
      time.sleep(1)
      # Validar si el boton borrar texto existe
      Boton = boton_existe(session,"wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                      ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
      if Boton:
        print("Botón Borrar presente")
        # presionar Boton borrar texto
        botonBorrar=session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                      ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
        botonBorrar.press()
        # Punto en el texto
        puntoentexto = session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:"
                                      "SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/cntlTEXT_EDITOR_0201/shellcont/shell")
        puntoentexto.text = "."
        # paso al siguiiente item de la solped
        #ventana=EncontrarDynpros(session)     
        #ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
        #print(f"ID Dinamico es: {ID}")
        #session.FindById(ID).press()
        #session.FindById(ID).setFocus()
        time.sleep(0.5)
        for i in range(2):
          pyautogui.hotkey("TAB")
          time.sleep(0.5)
        pyautogui.hotkey("enter")
      
      else:
        print("Botón Borrar NO está en esta vista")



NameError: name 'EncontrarDynpros' is not defined

In [41]:
import pyautogui


ventana=EncontrarDynpros(session)
ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
print(ID)
Posiciones=session.FindById(ID).Entries.Count
for i in range(Posiciones-2):
    print(i)
    ID2=ID
    ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
    combo=session.FindById(ID)
    item=session.FindById(ID).Entries(i)
    print(combo.Entries(i).value)
    combo.key=item.key
    if i>0:    
      session.FindById(ID2).selectedNode = "F01"
    # lista de posiciones de la solped
    #print(f"Navegando por la posición {i+1} de la Solped {solped}Esperando foco en pestaña textos..")
    # foco Select en la pestaña textos despues de imputacion 
    ventana=EncontrarDynpros(session)     
    ID= f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14"    
    print(f"ID Dinamico es: {ID}")
    session.FindById(ID).select()
    if i==0:
      ventana=EncontrarDynpros(session)     
      ID= f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14"         
    session.FindById(ID).setFocus()
    # ir a texto posicion con un TAB 
    pyautogui.hotkey("TAB")
    time.sleep(0.5)
    pyautogui.press('down') 
    for i in range(5):   
      # ir a pedido info con un flecha abajo
      pyautogui.press('down') 
      # entrar a editar texto con ctrl + enter
      pyautogui.hotkey("ctrl","enter")
      time.sleep(1)
      # Validar si el boton borrar texto existe
      Boton = boton_existe(session,"wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                      ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
      if Boton:
        print("Botón Borrar presente")
        # presionar Boton borrar texto
        botonBorrar=session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                      ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
        botonBorrar.press()
        # Punto en el texto
        puntoentexto = session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                                      ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:"
                                      "SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/cntlTEXT_EDITOR_0201/shellcont/shell")
        puntoentexto.text = "."
        # paso al siguiiente item de la solped
        #ventana=EncontrarDynpros(session)     
        #ID = f"wnd[0]/usr/subSUB0:SAPLMEGUI:{ventana}/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST"
        #print(f"ID Dinamico es: {ID}")
        #session.FindById(ID).press()
        #session.FindById(ID).setFocus()
        time.sleep(0.5)
        for i in range(2):
          pyautogui.hotkey("TAB")
          time.sleep(0.5)
        pyautogui.hotkey("enter")
      
      else:
        print("Botón Borrar NO está en esta vista")



wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB1:SAPLMEGUI:6000/cmbDYN_6000-LIST
0
[ 10 ] 80041940 , SERVICIO BICI-TALLER X BICICLET COL BICI
ID Dinamico es: wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
1
[ 20 ] 80001039 , ALQUILER DE CARPAS


AttributeError: Property 'FindById.selectedNode' can not be set.

In [21]:
ID="wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1:SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/cntlTEXT_TYPES_0200/shell"
combo = session.FindById(ID)
for attr in dir(combo):
    print(attr)

ChangeCheckbox
ClickLink
CollapseNode
DefaultContextMenu
DoubleClickItem
DoubleClickNode
DumpState
EnsureVisibleHorizontalItem
ExpandNode
FindAllByName
FindAllByNameEx
FindById
FindByName
FindByNameEx
FindNodeKeyByPath
GetAbapImage
GetAllNodeKeys
GetCheckBoxState
GetColumnCol
GetColumnHeaders
GetColumnIndexFromName
GetColumnNames
GetColumnTitleFromName
GetColumnTitles
GetFocusedNodeKey
GetHierarchyLevel
GetHierarchyTitle
GetIsDisabled
GetIsEditable
GetIsHighLighted
GetItemHeight
GetItemLeft
GetItemStyle
GetItemText
GetItemTextColor
GetItemToolTip
GetItemTop
GetItemType
GetItemWidth
GetListTreeNodeItemCount
GetNextNodeKey
GetNodeAbapImage
GetNodeChildrenCount
GetNodeChildrenCountByPath
GetNodeHeight
GetNodeIndex
GetNodeItemHeaders
GetNodeKeyByPath
GetNodeLeft
GetNodePathByKey
GetNodeStyle
GetNodeTextByKey
GetNodeTextByPath
GetNodeTextColor
GetNodeToolTip
GetNodeTop
GetNodeWidth
GetNodesCol
GetParent
GetPreviousNodeKey
GetSelectedNodes
GetSelectionMode
GetStyleDescription
GetSubNodesCol


In [39]:
session.FindById(ID).selectedNode = "F01"



In [ ]:
combo=session.FindById(ID)
item=session.FindById(ID).Entries(5)
combo.key=item.key

In [ ]:
import win32com.client
import subprocess
import time
import os
from Config.settings import RUTAS
from HU.HU1_LoginSAP import ObtenerSesionActiva
from Funciones.ValidacionM21N import find_dyn6000_combo,boton_existe,find_button_by_text
import pyautogui

session.findById('wnd[0]/shellcont/shell/shellcont[1]/shell[1]').setFocus()
time.sleep(2)
for i in range(6):   
    pyautogui.hotkey("ctrl","TAB")
    time.sleep(0.5)
pyautogui.press('right')
pyautogui.press('right')
pyautogui.press('enter')
for i in range(6):   
    pyautogui.hotkey("ctrl","TAB")
    time.sleep(0.5)

In [ ]:
import pyautogui
import time
def boton_existe(session,id):
    try:
        session.findById(id)
        return True
    except:
        return False
for i in range(3): 
    print(i)  
    pyautogui.press('down') 
    pyautogui.hotkey("ctrl","enter")
    time.sleep(0.5)
    Boton = boton_existe(session,"wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                    ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
            
    if Boton:
        print("Botón Borrar presente")
        # Boton borrar texto
        botonBorrar=session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                    ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
        botonBorrar.press()
        # Punto en el texto
        puntoentexto = session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:"
                                    "SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/cntlTEXT_EDITOR_0201/shellcont/shell")
        puntoentexto.text = "."
        pyautogui.hotkey("shift","TAB")
        time.sleep(0.5)
        pyautogui.press('down') 
        #session.sendVKey(2)
    else:
        print("Botón Borrar NO está en esta vista")

0
Botón Borrar presente
1
Botón Borrar NO está en esta vista
2
Botón Borrar NO está en esta vista


In [ ]:
for i in range(4):   
    pyautogui.hotkey("TAB")
    time.sleep(0.5)

def boton_existe(session,id):
    try:
        session.findById(id)
        return True
    except:
        return False
      
for i in range(3):   
    pyautogui.press('down') 
    pyautogui.hotkey("ctrl","enter")
    time.sleep(0.5)
    Boton = boton_existe(session,"wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                    ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
            
    if Boton:
        print("Botón Borrar presente")
        # Boton borrar texto
        botonBorrar=session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB"
                    ":SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/btnDELETE_0201")
        botonBorrar.press()
        # Punto en el texto
        puntoentexto = session.findById("wnd[0]/usr/subSUB0:SAPLMEGUI:0015/subSUB3:SAPLMEVIEWS:1100/subSUB2:SAPLMEVIEWS:1200/subSUB1"
                                    ":SAPLMEGUI:1301/subSUB2:SAPLMEGUI:1303/tabsITEM_DETAIL/tabpTABIDT14/ssubTABSTRIPCONTROL1SUB:"
                                    "SAPLMEGUI:1329/subTEXTS:SAPLMMTE:0200/subEDITOR:SAPLMMTE:0201/cntlTEXT_EDITOR_0201/shellcont/shell")
        puntoentexto.text = "."
        pyautogui.hotkey("shift","TAB")
        time.sleep(0.5)
        pyautogui.press('down') 
        #session.sendVKey(2)
    else:
        print("Botón Borrar NO está en esta vista")


      # Navegar hasta el boton flecha abajo para siguiente posicion
    
    for i in range(2):
    pyautogui.hotkey("TAB")
    time.sleep(0.5)
    pyautogui.press("enter")


Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista
Botón Borrar NO está en esta vista


In [ ]:


SelectGuiTab(session, "TABIDT14")
textPF = buscar_objeto_por_id_parcial(session, "cntlTEXT_TYPES_0200/shell")
nodo = "F01"
